In [2]:
#-*- coding: utf-8 -*-
from google.cloud import bigquery
from datetime import timedelta
from datetime import datetime
import argparse
import time
import sys, os
from time import sleep 

In [3]:
# Real배포시 수정사항
# 1. os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"
# 2. parse_parameters(): 안에 막는부분과 푸는 부분


os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"


In [4]:
def get_timedelta(sdatetime, edatetime):
    turnaround = edatetime - sdatetime
    total_seconds = int(turnaround.total_seconds())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    out = '{:02}:{:02}:{:02})'.format(hours,minutes,seconds)
    return out

In [5]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
def query_log(log_file_name, query_string):
    sql_log_filename = query_log_path+'/'+log_file_name
    with open(sql_log_filename, 'w') as file:
        file.write(query_string)
        file.close()


In [49]:
def get_totalrows(schema_id, table_id):
    start_now = datetime.now()
    query_string = """Select Count(*) total_rows
  From """+schema_id+"""."""+table_id
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    total_rows = [row['total_rows'] for row in query_job][0]
    #print('Execute >> [start:{}][elapsed:{}][{: 10} rows][row selected] [{}.{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()), total_rows, schema_id, table_id))
    return total_rows

In [46]:
def query_execute_create(destination_schema, destination_table, query_string, temp_table_name, step_name):
    start_now = datetime.now()
    # Query 내용 로그처리
    log_file_name = temp_table_name+'_'+step_name+'.sql'
    query_log(log_file_name, query_string)

    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()

    dataset_id=destination_schema
    table_id = destination_table
    table_ref = client.dataset(dataset_id).table(table_id)
    job_config.destination = table_ref
    job_config.write_disposition = 'WRITE_TRUNCATE'
    query_job = client.query(query_string, job_config=job_config)
    results = query_job.result()  
    # 1번째 방법
    #df_result = results.to_dataframe()
    #rows = results.total_rows
    # 2번째 방법
    # rows = len([i[0] for i in results])
    #rows = 0
    # 3번째 방법
    rows = get_totalrows(dataset_id, table_id)
    print('Execute >> [start:{}][elapsed:{}][{: 10} rows][{}] [{}.{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()), rows, step_name, destination_schema, destination_table))

    

In [8]:
def query_execute_dml(query_string, temp_table_name, step_name):
    start_now = datetime.now()
    # Query 내용 로그처리
    log_file_name = temp_table_name+'_'+step_name+'.sql'
    query_log(log_file_name, query_string)

    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query_string, job_config=job_config)
    results = query_job.result()  
    rows = query_job.num_dml_affected_rows
    print('Execute >> [start:{}][elapsed:{}][{: 10} rows][{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), 
                                                            get_timedelta(start_now, datetime.now()), rows, step_name))
    #print(results)

In [14]:
def query_execute_df(query_string):
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)

    return query_job.to_dataframe()

In [32]:
def step_01_01_query_string():
    query_string = """
Select a.AppID, a.CampaignID, a.ChannelID, a.ADID, 
        Upper(b.AppName) AppName, Upper(d.CampaignName) CampaignName, 
        Case When Upper(d.CampaignName) <> Upper('Organic') And Upper(e.channelName) = Upper('Organic') Then Upper('미입력') Else Upper(e.channelName) End channelName, 
        Case When Upper(d.CampaignName) <> Upper('Organic') And Upper(a.Location) = Upper('Organic') Then Upper('미입력') Else Upper(a.Location) End Location, 
        Case When Upper(c.schemaNM) = Upper('chacha') Then Upper('chachacha') When Upper(c.schemaNM) = Upper('pongpong') Then Upper('everypong') Else Upper(c.schemaNM) End as gamecode,
        b.GameKey, 
        Case When b.market = '02' Then Upper('iOS') Else Upper('AOS') End as OS,
        Case When Upper(g.schemaNM) = Upper('chacha') Then Upper('chachacha')
              When Upper(g.schemaNM) = Upper('pongpong') Then Upper('everypong') Else Upper(g.schemaNM) End as Channel_gamecode,
        f.Gamekey Channel_GameKey,
        d.mediatype, Upper(b.market) as market
    From Tier1_BI.code_info_ad_ver3_develop a 
        Inner Join Tier1_BI.code_info_app_ver3_develop b  ON a.AppID = b.AppID
        Inner Join Tier1_BI.code_dim_gamelist c  On Cast(b.gamekey as INT64) = c.gamekey
        Inner Join Tier1_BI.code_info_campaign_ver3_develop d  On a.CampaignID = d.CampaignID
        Inner Join Tier1_BI.code_info_channel_ver3_develop e  ON a.ChannelID = e.ChannelID
        Inner Join Tier1_BI.code_info_app_ver3_develop f  ON e.Adver_AppID = f.AppID
        Left Outer Join Tier1_BI.code_dim_gamelist g  On Cast(f.gamekey as INT64) = g.gamekey

    """
    print(query_string)
    return query_string

In [51]:
def step_01_02_query_string():
    query_string = """
        Delete From Tier1_BI.code_adkey_info_develop
        Where 1=1
        And upper(CampaignName) = upper('Netmarble Mass Marketing')
        And market not in ('01','02') 

    """
    print(query_string)
    return query_string

In [52]:
def main(meta_tuple):
    global date_key
    global query_log_path

    # Real적용시 수정
    #root_path = '/query_log/'
    root_path = '/home/mi_admin/airflow_python/query_log/'
    
    # Input Parameter
    date_key = meta_tuple

    program_name = 'tier1_bi_adver_adkey_info'

    query_log_path = root_path + program_name+'/'+datetime.now().strftime('%Y%m%d')
    ensure_dir(query_log_path)            # Directory Create
    date_key_dt = datetime.strptime(date_key, '%Y-%m-%d')
    temp_table_name = 'tier1_bi_adver_code_'+date_key.replace('-', '')+'_'+datetime.now().strftime('%Y%m%d%H%M%S')
    
    # 임시 고정
    #temp_table_name = 'tier1_bi_adver_code_20180901_112606'

    #########################################################
    #  Parameter, constant Display                          #
    #########################################################
    print('date_key        >>> {}                [{}]'.format(type(date_key), date_key))
    print('date_key_dt     >>> {}  [{}]'.format(type(date_key_dt), date_key_dt))
    print('query_log_path  >>> {}                [{}]'.format(type(query_log_path), query_log_path))
    print('temp_table_name >>> {}                [{}]'.format(type(temp_table_name), temp_table_name))
    print('')

    #########################################################
    # Adkey_info 생성                                       #
    #########################################################
    print('>> Adkey_info 생성')
    adkey_schema = 'Tier1_BI'
    adkey_table = 'code_adkey_info_develop'

    # 1. Source에서 Campaign관련 데이터를 테이블로 만든다.
    step_name = '01_01_adkey_info_create'
    query_execute_create(adkey_schema, adkey_table, step_01_01_query_string(), temp_table_name, step_name)

    # 2. ios, aos이외의 마켓 삭제하기
    step_name = '01_02_not_in_ios_aos_delete'
    query_execute_dml(step_01_02_query_string(), temp_table_name, step_name)
   
    
    result = 'success'
    return result

In [53]:
def parse_parameters():
    #################################################
    # 개발 완료되면 풀어야 함. !!!!!!!!!!!!!!!!!!!
    #################################################
    parser = argparse.ArgumentParser(description='시스템 구동에 필요한 파라미터를 입력하세요.')
    parser.add_argument("--date_key", help="배치작업 기준일자를 YYYY-MM-DD 형태로 입력하세요. 반드시 입력해야 하는 값입니다.", required=True)

    args = parser.parse_args()

    date_key = args.date_key

    try:
        if len(date_key.split('-')) != 3:
            raise ValueError('\'date\' parameter seems wrong. It must be in \'YYYY-MM-DD\' format.')
    except ValueError:
        raise

    print('[Start] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))        
    print('- Input parameter : --date_key = {}'.format(date_key))

    return date_key


In [55]:
if __name__ == '__main__':
    start_time = time.time()
    start_now = datetime.now()
    result = 0
    # sudo python3 tier1_bi_adver_code_20180903.py --date_key 2018-09-01
    result = main('2018-09-09')
    #result = main(parse_parameters())
    print('*'*100)
    print('[Complete] {} ~ {} total elapsed : {} seconds'.format(start_now.strftime("%Y-%m-%d %H:%M:%S"),datetime.now().strftime("%Y-%m-%d %H:%M:%S"), time.time()-start_time))



Select Count(*) as onoff From Tier0_BI.__TABLES__ Where table_id='Log_GlobalAd_20180909' 
date_key        >>> <class 'str'>                [2018-09-09]
date_key_dt     >>> <class 'datetime.datetime'>  [2018-09-09 00:00:00]
query_log_path  >>> <class 'str'>                [/home/mi_admin/airflow_python/query_log/tier1_bi_adver_code/20180910]
temp_table_name >>> <class 'str'>                [tier1_bi_adver_code_20180909_20180910160813]

Execute >> [start:2018-09-10 16:08:13][elapsed:00:00:06)][     57547 rows][01_adver_code_source_data] [temp.tier1_bi_adver_code_20180909_20180910160813_01_adver_code_source_data]
****************************************************************************************************
>> campaign 생성
Execute >> [start:2018-09-10 16:08:19][elapsed:00:00:04)][      1889 rows][02_01_campaign_groupby] [temp.tier1_bi_adver_code_20180909_20180910160813_campaign]
Execute >> [start:2018-09-10 16:08:24][elapsed:00:00:03)]                 [02_02_get_campaignid] (campaigni